# Chapter 6. Techniques for learning

여러 기법들을 통해 신경망 학습의 효율과 정확도를 높일 수 있다.
1. 가중치 매개변수의 최적값을 탐색하는 최적화 방법
2. 가중치 매개변수 초깃값 설정 방법
3. 하이퍼파라미터 설정 방법
4. 오버피팅 방지 기법: weight decay, dropout,...
5. batch normalization


In [3]:
import numpy as np
import sys,os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

---
### Parameter Optimization

In [2]:
# repeat SGD 
class SGD:
    def __init__(self,lr=0.01):
        self.lr=lr
    def update(self,params,grads):
        for key in params.keys():
            params[key]-=self.lr*grads[key]

# example code

# network=TwoLayerNet(...)
# optimizer=SGD()
# for i in range(10000):
#     ...
#     x_batch,t_batch=get_mini_batch(...)
#     grads=network.gradient(x_batch,t_batch)
#     params=network.params
#     optimizer.update(params,grads)
#     ...

위의 SGD class와 같이 최적화를 담당하는 클래스를 분리해 구현하면 기능을 모듈화하기 좋다. <br>
예를 들어 다른 최적화 기법에서도 update 라는 공통의 메소드를 갖도록 구현하면 추후 수정이 용이하다. 

__[SGD의 단점]__<br>
단순하고 구현이 쉽지만 문제에 따라 비효율적일 때가 있다. 예를 들어 심하게 굽이진 타원형 contour에서 SGD는 매우 비효율적으로 최적값을 찾아나간다. <br>
즉, anisotropy 함수(비등방성 함수, 방향에 따라 성질 (여기서는 기울기)이 달라지는 함수)에서는 탐색 경로가 비효율적이다. <br>
따라서 무작정 기울어진 방향으로 진행하는 것보다 더 나은 방안이 필요하다. 방안들로 모멘텀, AdaGrad, Adam 세 방법을 여기선 소개한다. 

__[Momentum]__<br>
$$ v=\alpha v - \eta \frac{\partial L}{\partial W} $$
$$ W=W+v $$
W는 갱신할 파라미터, v는 물리에서 말하는 속도에 해당한다. v 식은 기울기 방향으로 힘을 받아 물체가 가속된다는 물리 법칙을 나타낸다. 모멘텀은 공이 그릇의 바닥을 구르는 듯한 움직임을 보인다. <br>
alpha*v 항은 물체가 아무런 힘을 받지 않을 때 서서히 하강시키는 역할을 한다. 알파는 0.9 등의 값으로 설정. 물리에서의 지면 마찰이나 공기 저항에 해당한다. 

In [4]:
class Momentum:
    def __init__(self,lr=0.01,momentum=0.9):
        self.lr=lr
        self.momentum=momentum
        self.v=None
    def update(self,params,grads):
        if self.v is None:
            self.v={}
            for key,val in params.items():
                self.v[key]=np.zeros_like(val)
        
        for key in params.keys():
            self.v[key]=self.momentum*self.v[key]-self.lr*grads[key]
            params[key]+=self.v[key]


모멘텀의 갱신 경로는 공이 그릇 바닥을 구르듯 움직인다. SGD와 비교하면 지그재그 정도가 덜함을 알 수 있다. <br>
이는 x축의 힘은 아주 작지만 방향은 변하지 않아서 한 방향으로 일정하게 가속하기 때문이다. 반대로 y축의 힘은 크지만 위아래로 번갈아 받아서 상충하여 y축 방향의 속도는 안정적이지 않다. <br>
전체적으로 SGD보다 x축 방향으로 빠르게 다가가 지그재그 움직임이 줄어드는 형상을 보인다. 

__[AdaGrad]__
$$h=h+\frac{\partial L}{\partial W} \odot \frac{\partial L}{\partial W} $$
$$W=W-\eta \frac{1}{\sqrt{h}} \frac{\partial L}{\partial W} $$

학습률 값이 너무 작으면 학습 시간이 길어지고 너무 크면 발산하여 학습이 제대로 안이루어진다. 이 학습률을 정하는 효과적 기술로 learning rate decay가 있다. <br>
이는 학습을 진행하면서 학습률을 점차 줄여가는 방법이다. 처음엔 크게 학습하다 조금씩 작게 학습한다. 실제 신경망 학습에 자주 쓰인다. <br>
가장 간단한 방법으로 매개변수 '전체'의 학습률 값을 일괄적으로 낮추는 것이다. 이를 더욱 발전시킨 것이 AdaGrad<br>
AdaGrad는 '각각의' 매개변수에 '맞춤형' 값을 만들어준다. <br>
AdaGrad는 개별 매개변수에 적응적으로 (adaptive) 학습률을 조정하면서 학습을 진행한다. <br>
변수 h는 기존 기울기 값을 제곱하여 계속 더해준다. (동그라미 기호는 행렬의 원소별 곱셈을 의미) 그리고 매개변수를 갱신할 때 1/sqrt(h)를 곱해 학습률을 조정한다.<br>
매개변수의 원소 중에서 많이 움직인 (크게 갱신된) 원소는 학습률이 낮아진다는 뜻인데 다시 말해 학습률 감소가 매개변수의 원소마다 다르게 적용됨을 뜻한다.

AdaGrad는 과거의 기울기를 제곱하여 계속 더해간다. 그래서 학습을 진행할수록 갱신 강도가 약해진다. 실제로 무한히 계속 학습하면 어느순간 갱신량이 0이 된다. 이 문제를 개선하고자 RMSProp 이라는 방법이 있다. 이는 과거의 모든 기울기를 균일하게 더해가는 것이 아니라 먼 과거의 기울기는 서서히 잊고 새로운 기울기 정보를 크게 반영한다. 이를 exponential moveing average (EMA)라 하여, 과거 기울기의 반영 규모를 기하급수적으로 감소시킨다. 

In [5]:
class AdaGrad:
    def __init__(self,lr=0.01):
        self.lr=lr
        self.h=None
    def update(self,params,grads):
        if self.h is None:
            self.h={}
            for key,val in params.items():
                self.h[key]=np.zeros_like(val)
        for key in params.keys():
            self.h[key]+=grads[key]*grads[key]
            params[key]-=self.lr*grads[key]/(np.sqrt(self.h[key])+1e-7)
            

작은 값을 더해줌으로써 0으로 나누는 사태 막아줌


최솟값을 향해 효율적으로 움직인다. y축 방향은 기울기가 커서 처음엔 크게 움직이지만 그 큰 움직임에 비례해 갱신 정도도 큰 폭으로 작아지도록 조정된다. 따라서 y축 방향으로 갱신 강도가 빠르게 약해지고 지그재그 움직임이 줄어든다. 

__[Adam]__<br>
복잡하지만 간단하게 Momentum + AdaGrad 융합한 방법(엄밀히 완전 융합은 아님)<br>
매개변수 공간을 효율적으로 탐색해주고 하이퍼파라미터의 편향 보정이 진행된다. <br>
Adam은 하이퍼파라미터 3개를 설정한다. 학습률, 일차 모멘텀용계수, 이차 모멘텀용 계수. 논문에 따르면 default로 일차계수는 0.9, 이차계수는 0.999 이며 이 값이면 많은 경우에 좋은 결과를 얻는다 한다. <br>
구현은 다소 복잡하니 생략<br>

사용한 기법에 따라 갱신 경로가 다르며 어떤 기법을 사용할 지는 풀어야 할 문제가 무엇이냐에 따라 달라진다. 또한 하이퍼파라미터와 신경망의 구조를 어떻게 설정하느냐에 따라서도 결과가 바뀐다. <br>
일반적으로 SGD보다 다른 세 기법이 빠르게 학습하고 때로는 최종 정확도도 높게 나타난다. 

---
### Weight Initialization

__Weight decay__<br>
가중치 감소는 오버피팅을 억제해 범용 성능을 높이는 테크닉이다. <br>
간단히 말해 가중치 매개변수의 값이 작아지도록 학습하는 방법이다. 가중치 값을 작게 해 오버피팅이 일어나지 않게 한다. <br><br>
__가중치 초깃값을 모두 0 (또는 동일 값) 으로 설정하면 안되는 이유__<br>
가중치의 초깃값을 모두 0으로 설정하면 학습이 올바로 이뤄지지 않는다. 정확히는 가중치를 균일한 값으로 설정해서는 안된다. 이유는 backpropagation 에서 모든 가중치의 값이 똑같이 갱신되기 때문이다. <br>
예를 들어 2층 신경망에서 첫번째 두번째 층 가중치가 0이라 하면, 순전파때 입력층 가중치가 0이기 때문에 두번째 층의 뉴런에 모두 같은 값이 전달된다. 두번째 층 모든 뉴런에 같은 값이 입력된다는 것은 역전파시 두번째 층의 가중치가 모두 똑같이 갱신된다는 말이다. 따라서 가중치들은 같은 초깃값에서 시작하고 갱신을 거쳐도 여전히 같은 값을 유지한다. 이는 가중치를 여러개 갖는 의미를 사라지게 한다. <br>
가중치가 고르게 되어버리는 상황을 막으려면 초깃값을 무작위로 설정해야한다. 

__가중치의 초깃값에 따라 은닉층 활성화값들이 어떻게 변화하는지 알아보자.__ <br>
가중치 분포의 표준편차가 상대적으로 크게 가중치를 초기화 했을때 (W=np.random.randn(node_num,node_num)*1) <br>
각 층의 활성화 값들은 0과 1에 치우쳐 분포한다. 이는 활성함수인 시그모이드 함수가 그 출력이 0 또는 1에 가까워지자 미분이 0에 다가가기 때문이다. 따라서 데이터가 0과1에 치우쳐 분포하게 되면 역전파의 기울기 값이 점점 작아지다가 사라진다. 이것은 gradient vanishing 이라 알려진 문제이다. 층이 깊은 딥러닝에서는 기울기 소실이 심각한 문제가 될 수 있다. <br>
가중치 분포의 표준편차가 상대적으로 작게 가중치를 초기화 했을때 (W=np.random.randn(node_num,node_num)*0.01) <br>
활성화 값이 0.5 부근에 집중되어있다. 기울기 소실 문제는 일어나지 않지만 활성화 값이 치우쳤다는 것은 표현력 관점에서 큰 문제이다. 다수의 뉴런이 거의 같은 값을 출력하고 있으니 뉴런을 여러 개 둔 의미가 없어진다. 예를 들어 뉴런 100개가 거의 같은 값을 출력한다면 뉴런 1개와 별반 다를게 없다는 것이다. 그래서 활성화값들이 치우치면 표현력을 제한한다는 관점에서 문제가 된다.<br>
따라서 각 층의 활성화 값은 적당히 고루 분포되어야한다. 층과 층 사이에 적당하게 다양한 데이터가 흐르게 해야 신경망 학습이 효율적으로 이뤄지기 때문이다. 반대로 치우친 데이터가 흐르면 기울기 소실이나 표현력 제한 문제에 빠져서 학습이 잘 이뤄지지 않는 경우가 생긴다. 

__Xavier initialization (활성함수: sigmoid, tanh)__<br>
사비에르 글로로트와 요슈아 벤지오가 권장하는 가중치 초깃값. 현재 Xavier 초깃값은 일반적인 딥러닝 프레임워크들이 표준적으로 이용한다. <br>
이 방법은 각 층의 활성화값들을 광범위하게 분포시킬 목적으로 가중치가 적절한 분포를 갖도록 한다. 결론은 앞 계층의 노드가 n개라면 표준편차 1/(sqrt(n)) 인 분포를 초깃값으로 사용하면 된다는 것이다. <br>
xavier 초깃값을 사용하면 앞층의 노드가 많을수록 대상 노드의 초깃값으로 설정하는 가중치가 좁게 퍼지게 된다. <br>
층이 깊어지면서 활성화값 분포가 형태는 다소 일그러지지만 확실히 넓게 분포됨을 알 수 있다. 각 층에 흐르는 데이터가 적당히 퍼져 있으므로 시그모이드 함수의 표현력도 제한받지 않고 학습이 효율적으로 이뤄질 것으로 기대된다. <br>
일그러짐 문제는 sigmoid 대신 tanh 함수를 사용하면 개선된다. tanh와 sigmoid함수는 모양이 둘다 s자 이지만 tanh 는 원점대칭인 함수, sigmoid는 (0,0.5)에서 대칭인 함수이다. 활성화 함수 용으로는 원점 대칭인 함수가 바람직하다고 알려져 있다. 

__He initialization (활성함수: ReLU)__<br>
xavier는 활성함수가 선형인 것을 전제로 이끈 결과이다. 이는 좌우대칭인 sigmoid, tanh함수가 중앙 부근에서 선형인 함수로 볼 수 있기 때문에 적용가능했다. 반면 ReLU는 카이밍 히가 만든 He 초깃값을 이용하는 것을 권장된다. 이는 앞 계층 노드가 n개 일때, 표준편차 sqrt(2/n)인 정규분포를 사용한다. 직감적 해석으로, ReLU는 음의 영역이 0이라서 더 넓게 분포시키기 위해 xavier보다 2배의 계수가 필요하다고 해석할 수 있겠다. 

__Initialization 정리__<br>
활성함수가 sigmoid, tanh 등의 S자 모양 곡선: Xavier initialization<br>
활성함수가 ReLU: He initialization<br>


---
### Batch Normalization

각 층에서의 활성화 값이 적당히 분포되도록 조정한다. <br>
1. 학습을 빨리 진행할 수 있다. (학습 속도 개선)
2. 초깃값에 크게 의존하지 않는다.
3. 오버피팅을 억제한다. (드롭아웃 등의 필요성 감소)


데이터 분포를 정규화하는 '배치 정규화 계층' 을 신경망에 삽입한다. 활성화 함수 계층 앞 또는 뒤에 삽입한다. <br>
학습 시 미니배치를 단위로 정규화 시킨다. 즉 미니배치 입력 데이터 분포가 평균0, 분산1이 되도록 정규화한다. 이 처리를 활성화 함수 앞 (또는 뒤)에 삽입함으로써 데이터 분포가 덜 치우치게 할 수 있다. <br>
아래 식에서 미니배치 m개의 데이터의 집합에 대한 평균과 분산을 구하고 데이터를 새로 정규화 시킨다.

$$\mu_B=\frac{1}{m}\sum_{i=1}^{m}x_i$$
$${\sigma}^2_B=\frac{1}{m}\sum_{i=1}^{m}(x_i-\mu_B)^2$$
$$\hat{x_i}=\frac{x_i-\mu_B}{\sqrt{\sigma^2_B+\epsilon}}$$

또한 배치 정규화 계층마다 이 정규화된 데이터에 고유한 scale 과 shift 변환을 수행한다. 
$$ y_i = \gamma \hat{x_i}+\beta $$

두 값은 처음에는 감마=1, 베타=0 부터 시작하고 학습하면서 적합한 값으로 조정해간다. 자세한 설명은 Frederik Kratzert의 블로그 참초<br>
배치 정규화는 이제 대세이며, 이를 이용하지 않을 경우 초깃값이 잘 분포되어 있지 않으면 학습을 잘 진행하기 어렵다.<br>
다시 말하지만 학습이 빨라지고 가중치 초깃값에 크게 의존하지 않아도 된다. 

---
### Overfitting problem

오버피팅: 신경망이 훈련 데이터에만 지나치게 적응되어 그 외 데이터에 제대로 대응하지 못하는 상태<br>
주로 다음의 두 경우에 일어난다. 
    * 매개변수가 많고 표현력이 높은 모델
    * 훈련 데이터가 적음

__weight decay__<br>
예로부터 많이 이용해온 오버피팅 방지 기법. 학습 과정에서 큰 가중치에 대해서는 그에 상응하는 큰 페널티를 부과하여 오버피팅을 억제한다. 원래 오버피팅은 가중치 매개변수의 값이 커서 발생하는 경우가 많기 때문이다. <br>
예를 들어 손실함수에 L2 가중치 제곱을 더한다. (정규화) 이 때 람다, 정규화 세기를 조절하는 하이퍼파라미터가 도입된다. 람다를 크게 할수록 큰 가중치에 대한 페널티가 더 커진다. 

__Dropout__<br>
신경망 모델이 복잡해지면 가중치 감소만으로 대응하기 어렵다. 이때 흔히 드롭아웃이라는 기법을 이용한다. <br>
드롭아웃은 뉴런을 임의로 삭제하면서 학습하는 방법이다. 훈련 시 은닉층의 뉴런을 무작위로 골라 삭제한다. 삭제된 뉴런은 신호를 전달하지 않게 된다. <br>
훈련 때에는 데이터를 흘릴 때마다 삭제할 뉴런을 무작위로 선택하고, 시험 때는 모든 뉴런에 신호를 전달한다.<br>


In [6]:
class Dropout:
    def __init__(self,dropout_ratio=0.5):
        self.dropout_ratio=dropout_ratio
        self.mask=None
    def forward(self,x,train_flg=True):
        if train_flg:
            self.mask=np.random.rand(*x.shape)>self.dropout_ratio
            return x*self.mask
        else:
            return x*(1.0-self.dropout_ratio)
    def backward(self,dout):
        return dout*self.mask

순전파 때 신호를 통과시키는 뉴런은 역전파 때도 신호를 그대로 통과시키고, 순전파 때 통과시키지 않은 뉴런은 역전파 때도 신호를 차단한다. <br>
드롭아웃을 이용하면 표현력을 높이면서도 오버피팅을 억제할 수 있다.

기계학습에선 앙상블 학습을 애용한다. 이는 개별적으로 학습시킨 여러 모델의 출력을 평균 내어 추론하는 방식이다. 앙상블 학습을 수행하면 신경망의 정확도가 몇% 정도 개선된다는 것이 실험적으로 알려져 있다. <br>
앙상블 학습은 드롭아웃과 밀접하다. 드롭아웃이 학습 때 뉴런을 무작위로 삭제하는 행위를 매번 다른 모델을 학습시키는 것으로 해석 할 수 있기 때문이다. 그리고 추론 때는 뉴런의 출력에 삭제한 비율을 곱함으로써 앙상블 학습에서 여러 모델의 평균을 내는 것과 같은 효과를 얻는다. 즉 드롭아웃은 앙상블 학습과 같은 효과를 (대략) 하나의 네트워크로 구현했다고 생각할 수 있다. 

---
### Hyperparameter 


각 층의 뉴런수, 배치 크기, learning rate, weight decay 등의 하이퍼파라미터 값을 적절히 설정하는 것은 중요하다. <br>
하이퍼파라미터 값을 최대한 효율적으로 탐색하는 방법으로 validation data 을 사용한다. 하이퍼파라미터 조정용 데이터이며 하이퍼파라미터의 적절성을 평가하는 데이터이다. <br>
훈련데이터->매개변수 학습, 검증데이터->하이퍼파라미터 성능평가, 시험데이터->신경망의 범용성능평가<br>
하이퍼파라미터 최적화는 최적값이 존재하는 범위를 조금씩 줄여나감으로서 진행한다. 대략적인 범위 설정 후 그 범위에서 무작위로 샘플링 후 그 값으로 정확도를 평가한다. 정확도를 살피면서 이 작업을 반복하며 하이퍼파라미터의 최적 값의 범위를 좁혀나간다. 샘플링 시 grid search 보다는 random search로 탐색하는 편이 좋은 결과를 낸다고 알려져있다. <br>
이러한 최적화 방법은 다소 실용적인 방법으로 수행자의 직관에 의존한다. 좀 더 세련된 기법으로는 Bayesian optimization이 있으며 더 엄밀하고 효율적으로 최적화를 수행한다. 

---
### [6장 정리]

* 매개변수 갱신 방법: SGD, momentum, AdaGrad, Adam, ...
* 가중치 초깃값을 정하는 방법은 올바른 학습을 하는데 매우 중요하다. : Xavier, He,...
* Batch normalization을 통해 빠른 학습을 진행하며 초깃값에 영향을 덜 받게 된다.
* Overfitting 방지 정규화 기술로 weight decay, dropout 이 있다.
* hyperparameter 탐색은 최적 값이 존재할 법한 범위를 점차 좁히면서 하거나 Bayesian optimization 방법을 이용한다. 